In [4]:
a=2
b=4
c=a+b
print(c)

6


In [ ]:
# 3.1 OpenAI 연결
# openAI API를 사용하면 memory등의 이점이 없음, 코드작성자가 다 처리해야함
# langchain을 사용하면 다양한 세팅들을 알아서 처리해줌
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm=OpenAI()
chat=ChatOpenAI(temperature=0.1)
#max_token
#temperature 높을수록 창의적임
a= llm.predict("how many planet are there?")
b= chat.predict("how many planet are there?")

In [ ]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage

message=[
    SystemMessage(content="You ara geography expert. And you only reply in Koean"),
    AIMessage(content="내이름은 테스트 봇이야"),
    HumanMessage(content="맥시코와 태국사이의 거리는 어떻게되나 그리고 너의 이름은 뭐야")
]
chat.predict_messages(message)

In [ ]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage

message=[
    SystemError(content="You ara geography expert. And you only reply in {language}"),
    AIMessage(content="내이름은 {name} 봇이야"),
    HumanMessage(content="{country_a}와 {country_b} 사이의 거리는 어떻게되나 그리고 너의 이름은 뭐야?")
]
chat.predict_messages(message)

In [ ]:
# 3.2 langchain prompt 사용법
# prompt는 llm과의 소통창구로 prompt 성능에 따라 llm의 답변 수준이 정해짐
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)
prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)


template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)


chat.predict_messages(prompt)

In [ ]:
# 3.3 Output Parser
# Output Parser의 사용이유 : LLM response 변형용(llm은 보통 text로 답함)
# answered with a comma separated list등을 system message에 추가해서 답변형태를 강제 할 수 있음
# chain = template+llmModel+outputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

# chain 끼리 결합도 가능
chain = template | chat | CommaOutputParser()
chain_2 = template_2 | chat_2 | CommaOutputParser2
all_chain=chain|chain_2|outputparser3


In [11]:
# 3.4
# chain = prompt+retriever+llm+tool+parser

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat

# final_chain=chef_chain|veg_chain 
# chef에서 얻은 recipe(output)이 veg의 input으로사용
# 이 final_chain을 쓰면 invoke를 두번해야함
# 아래녀석도 코드상으로만 1번호출이지 실제 2번호출함
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})